In [ ]:
import itertools
import pathlib
import sys

import matplotlib as mpl
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np

sys.path.append(str(pathlib.PurePath('..', '..', 'src')))
from linear_geodesic_optimization.data import input_network, tomography
from linear_geodesic_optimization import plot

In [ ]:
path_probes = pathlib.PurePath('probes.csv')
path_links = pathlib.PurePath('links.csv')
graph = input_network.get_graph_from_csvs(path_probes, path_links, clustering_distance=500000, should_compute_curvatures=False, directed=True, symmetrize=True)

routes = tomography.get_shortest_routes(graph, 'rtt')
traffic_matrix = tomography.compute_traffic_matrix(graph, routes, 'throughput')

In [ ]:
graph = input_network.compute_ricci_curvatures(
    graph, weight_label='throughputs',
    routes=routes, traffic_matrix=traffic_matrix
)

fig, ax = plt.subplots(1, 1, facecolor='#808080')
fig.set_dpi(200)
plot.get_network_plot(graph, ax=ax)
plt.show()

In [ ]:
# Compute shortest path lengths of actual routes
lengths = {}
for source, routes_source in routes.items():
    for destination, route in routes_source.items():
        length = 0.
        for u, v in itertools.pairwise(route):
            length += graph.edges[u, v]['rtt']
        lengths[route[0], route[-1]] = length

In [ ]:
# For each route, remove each link and check how it influences the
# route's length
lengths_with_removals = {}
for source, routes_source in routes.items():
    for destination, route in routes_source.items():
        lengths_with_removals_source_destination = {}

        for u, v in itertools.pairwise(route):
            # Remove the link
            u_v_data = graph.edges[u, v]
            graph.remove_edge(u, v)

            # Compute the shortest path length
            lengths_with_removals_source_destination[u, v] \
                = nx.shortest_path_length(graph, u, v, 'rtt')

            # Restore the link
            graph.add_edge(u, v, **u_v_data)

        lengths_with_removals[source, destination] \
            = lengths_with_removals_source_destination

In [ ]:
x = np.array([
    graph.edges[u, v]['ricciCurvature']
    for (source, destination), lengths_with_removals_source_destination in lengths_with_removals.items()
    for (u, v) in lengths_with_removals_source_destination
])
y = np.array([
    length_with_removal / lengths[u, v]
    for (source, destination), lengths_with_removals_source_destination in lengths_with_removals.items()
    for (u, v), length_with_removal in lengths_with_removals_source_destination.items()
])
colormap = mpl.colormaps['RdBu']
traffic_max = max(traffic_matrix.values())
colors = np.array([
    colormap(traffic_matrix[source, destination] / traffic_max)
    for (source, destination), lengths_with_removals_source_destination in lengths_with_removals.items()
    for (u, v) in lengths_with_removals_source_destination
])

plt.scatter(x, y, c=colors)
plt.colorbar(label='Traffic')
plt.xlabel('Ricci Curvature')
plt.ylabel('Relative Change in Route Length')
plt.show()